In [0]:
%run ./01-config

In [0]:
class Setup():
    def __init__(self, db_name):
        conf = Config()
        self.db_name = db_name
        self.catalog_name = conf.catalog_name
        self.storage_account = conf.storage_account + self.db_name + "/"
        self.initialized = False

    def create_db(self):
        spark.catalog.clearCache()
        print(f"Creating the database {self.catalog_name}.{self.db_name}...", end=" ")
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.catalog_name}.{self.db_name}")
        self.initialized = True
        print("Done")
    
    def cleanup(self):
        if spark.sql(f"SHOW DATABASES IN {self.catalog_name}").filter(f"databaseName = '{self.db_name}'").count() == 1:
            print(f"Dropping the database {self.catalog_name}.{self.db_name}...", end='')
            spark.sql(f"DROP DATABASE IF EXISTS {self.catalog_name}.{self.db_name} CASCADE")
            print("Done")
        print(f"Deleting Files from Storage Account from {self.storage_account}")
        items = dbutils.fs.ls(self.storage_account)
        for item in items:
            dbutils.fs.rm(item.path, True)
            print(f"✅ Deleted: {item.path}")

        print(f"✅ All files and subfolders inside {self.storage_account} are deleted")
        self.initialized = False
